<h1> Projet AXA </h1>

<h2> Raphaël Meudec, Guillaume Richard, Antoine Simoulin </h2>

# Requirement

In [1]:
%matplotlib inline
import os
import glob
import numpy as np
from scipy import io
import matplotlib.pyplot as plt
import pandas as pd
import matplotlib
from datetime import datetime
import error_functions as ef
from dateutil import relativedelta
from workalendar.europe import France
# pip install workalendar
import statsmodels.api as sm
import matplotlib as mpl
import numdifftools as nd

/Users/Antoine/anaconda2/lib/python2.7/site-packages/pandas/computation/__init__.py:19: UserWarning: The installed version of numexpr 2.4.4 is not supported in pandas and will be not be used

  UserWarning)


In [2]:
# colors for plot
blue_light = '#029aed'
orange_med = '#ff5722'
green_light = '#63a600'
gray_light = '#666666'
blue_AXA = '#103184'
red_AXA = '#ff1821'
cal = France()
# set colors for graphs
mpl.rcParams['axes.color_cycle'] = [blue_AXA, red_AXA, gray_light]

/Users/Antoine/anaconda2/lib/python2.7/site-packages/matplotlib/__init__.py:878: UserWarning: axes.color_cycle is deprecated and replaced with axes.prop_cycle; please use the latter.
  warnings.warn(self.msg_depr % (key, alt_key))


In [3]:
# create DataFrame with public holliday in France for years 2011, 2012, 2013, 2014
holiday_map = pd.DataFrame()
for year in [2011,2012,2013,2014]:
    holiday_map_temp = pd.DataFrame(cal.holidays(year))
    holiday_map_temp = holiday_map_temp.set_index([0])
    holiday_map = pd.concat([holiday_map, holiday_map_temp], ignore_index=False)
# holiday_map.head()  

In [4]:
# loading the train data
data = pd.read_csv('data/train.csv', sep=";", parse_dates=['DATE'], index_col = ['DATE'], nrows = 3*1e4)
data['DATE'] = data.index
# data.head()

In [5]:
# timestamp exctraction
def splitDatetime(data) :
    datatime = pd.DatetimeIndex(data.DATE)
    data['year'] = datatime.year
    data['month'] = datatime.month
    data['day'] = datatime.day
    data['hour'] = datatime.hour
    data['min'] = datatime.minute
    data['dayweek'] = datatime.weekday
    data['workingday'] = (datatime.weekday < 6).astype(int)
    data['holiday'] = data.index.isin(holiday_map.index).astype(int)
    data['night'] = (datatime.hour < 7).astype(int)
    return data

data = splitDatetime(data)

# Label definition

In [6]:
def format_label():
    # pivot table on data for ASS_ASSIGNMENT columnwise
    X = pd.DataFrame()
    X = data[['year','month','day','hour','ASS_ASSIGNMENT','CSPL_CALLS','min','DATE']]
    df1 = X.pivot_table(index = ['DATE'], columns = ['ASS_ASSIGNMENT'], values = ['CSPL_CALLS'], aggfunc=np.sum)
    # print(df1.shape)
    # when data for this date an categorie not available, fill with 0
    df1.fillna(0, inplace=True)
    # df1.head()

    # creating labels, ASS_ASSIGNMENT are presented columnwise thanks to pivot table above
    y_df = pd.DataFrame()
    for cat in data.ASS_ASSIGNMENT.unique() :
        y_df[cat] = df1['CSPL_CALLS'][cat]
    y_df.head()
    
    return y_df
    
y_df = format_label()

/Users/Antoine/anaconda2/lib/python2.7/site-packages/dateutil/parser.py:98: UnicodeWarning: Unicode equal comparison failed to convert both arguments to Unicode - interpreting them as being unequal
  while nextchar == '\x00':
/Users/Antoine/anaconda2/lib/python2.7/site-packages/dateutil/parser.py:123: UnicodeWarning: Unicode equal comparison failed to convert both arguments to Unicode - interpreting them as being unequal
  elif nextchar == '.':


In [7]:
def format_train(y_df):
    
    # formatting train data
    X_train = pd.DataFrame(index = y_df.index)
    X_train['DATE'] = y_df.index
    X_train = splitDatetime(X_train)
    X_train.drop('DATE', axis=1, inplace=True)
    
    date_min = X_train.index.min()
    date_max = X_train.index.max()
    X_train_range = relativedelta.relativedelta(date_max, date_min)

    # print train data range
    print('train data first date : %s' %date_min)
    print('train data last date  : %s' %date_max)
    print('train data range : %s years, %s months and %s days\n' %(X_train_range.years, X_train_range.months,X_train_range.days))
    # X_train.head()
    
    return X_train

# Features definition

In [8]:
# function to split data for CV purposes
# algorithm is test on last week of the randomly choosen train data set

def split_data(X_train):
    
    X_train_split = X_train
    # the subset contain at leat 50% of the original data
    s = np.random.uniform(X_train_split.shape[0]*0.9,X_train_split.shape[0],1).astype(int)
    X_train_split.drop(X_train_split.index[s:], inplace=True)

    date_min = X_train.index.min()
    date_max = X_train.index.max()
    X_train_range = relativedelta.relativedelta(date_max, date_min)

    # print subset range
    print('train data first date : %s' %date_min)
    print('train data last date  : %s' %date_max)
    print('train data range : %s years, %s months and %s days\n' %(X_train_range.years, X_train_range.months,X_train_range.days))
    # X_train.head()
    
    # Last week of the subset is used for cross validation purposes
    X_train_split_CV = X_train_split.last('7d')
    date_min_CV = X_train_split_CV.index.min()
    date_max_CV = X_train_split_CV.index.max()
    X_CV_range = relativedelta.relativedelta(date_max_CV, date_min_CV)

    # print CV subset week range
    print('CV data first date : %s' %date_min_CV)
    print('CV data last date  : %s' %date_max_CV)
    print('CV data range : %s years, %s months and %s days' %(X_CV_range.years, X_CV_range.months,X_CV_range.days))
    # X_train_CV.head()
    
    return X_train_split, X_train_split_CV


# Feature extractor

In [9]:
import numpy as np
import pandas as pd
from sklearn import preprocessing as pre

class FeatureExtractor(object):
    def __init__(self):
        pass

    def fit(self, X_df, y_df):
        pass
    
    def transform(self, X_df):
        return X_df

## Regressor loss definition

In [16]:
# this is log likelihood loss
def custom_obj(y_true, y_pred):
    grad = 0.1*exp(y_true-y_pred)
    hess = 0.1**2*np.exp(0.1*(y_true-y_pred))
    grad = xgb.DMtarix(grad)
    hess = xgb.DMatrix(hess)
    return grad, hess


def logregobj(preds, dtrain):
    labels = dtrain.get_label()
    preds = 1.0 / (1.0 + np.exp(-preds))
    grad = preds - labels
    hess = preds * (1.0-preds)
    return grad, hess

# Regressor

In [17]:
from sklearn.base import BaseEstimator
import xgboost as xgb
from sklearn.pipeline import Pipeline

class Regressor(BaseEstimator):
    
    def custom_obj(y_true, y_pred):
        grad = 0.1*exp(y_true-y_pred)
        hess = 0.1**2*np.exp(0.1*(y_true-y_pred))
        # grad = xgb.DMtarix(grad)
        # hess = xgb.DMatrix(hess)
        return grad, hess
    
    def __init__(self):
        
        self.reg_std = Pipeline([
            ('xgb', xgb.XGBRegressor(
                objective = custom_obj#'reg:linear',##
                #learning_rate = 0.1,
                #n_estimators = 100,
                #max_depth = 2,
                #min_child_weight = 1,
                #gamma = 0.1,
                #subsample = 0.9,
            ))
        ])

    def fit(self, X, y):
        return self.reg_std.fit(X, y)
        #return xgb.train(param, dtrain, num_round, watchlist, obj=softkappaobj, feval=evalerror)

    def predict(self, X, cat):
        return self.reg_std.predict(X)
    
    def predict_proba(self, X):
        return self.reg.predict_proba(X)

In [18]:
def linex_err(y_pred, y_train):
    return np.sum(0.1*np.exp(y_train-y_pred)-0.1*(y_train-y_pred))-1

# Cross validation Module

In [19]:
from sklearn.model_selection import ShuffleSplit
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.model_selection import TimeSeriesSplit
from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb


def train_test_model_clf(X_df, y_df, skf_is, FeatureExtractor, Regressor, GS):
    
    y_train_reg = {}
    y_test_reg = {}
    y_pred_reg = {}
    reg = {}
    error = 0
    
    for cat in data.ASS_ASSIGNMENT.unique():
        if cat in ['Téléphonie'] :

            # Spliting data for cross validation
            train_is, test_is = skf_is
            
            # test/train definition
            X_train_df = X_df.iloc[train_is].copy()
            y_train_df = y_df.iloc[train_is].copy()
            X_test_df = X_df.iloc[test_is].copy()
            y_test_df = y_df.iloc[test_is].copy()
            
            
            # for téléphonie only use last 6 month because data range to vary a lot each year
            if cat in ['Téléphonie','Tech. Axa','CAT']:
                X_train_df = X_train_df.last('3m')
                y_train_df = y_train_df.last('3m')

            # label category definition
            y_train_reg[cat] = y_train_df[cat].values
            y_test_reg[cat] = y_test_df[cat].values

            # Features extraction (no modification of data in this case)
            fe_reg = FeatureExtractor()
            fe_reg.fit(X_train_df, y_train_df)
            X_train_array_reg = fe_reg.transform(X_train_df)
            X_test_array_reg = fe_reg.transform(X_test_df)

            # Train
            # regressors initialisation
            reg[cat] = Regressor()

            # fitting model
            reg[cat].fit(X_train_array_reg, y_train_reg[cat])

            # Test
            y_pred_reg[cat] = np.round(np.maximum(reg[cat].predict(X_test_array_reg,cat),0),0)            
            error_tmp = linex_err(y_pred_reg[cat], y_test_reg[cat])
            error += error_tmp
            print('error %s = %.1f' %(cat.decode('utf-8'),error_tmp))
            
            # plot figure for predicted week
            y_test_plt = pd.DataFrame(index = y_test_df.index)
            y_test_plt[cat.decode('utf-8')] = y_test_reg[cat]
            y_pred_plt = pd.DataFrame(index = y_test_df.index)
            y_pred_plt[cat.decode('utf-8')] = y_pred_reg[cat]
            
            fig = plt.figure(figsize=[15,4])
            plt.style.use('ggplot')
            plt.plot(y_test_plt[cat.decode('utf-8')], color = blue_AXA)
            plt.plot(y_pred_plt[cat.decode('utf-8')], linestyle = 'dashed', color=red_AXA, linewidth=2)
            plt.title('error on predected week for %s (error = %.1f)' %(cat.decode('utf-8'),error_tmp))
            plt.legend(['true','pred'],loc='best')
            fig.savefig('plots/Pred_appels_categorie_%s.png' %(cat),bbox_inches='tight')
            
            #print('\n------------------------------------------------')
        #else:
            #print("pass.")
            #print('------------------------------------------------')
    k = X_train_df.shape[0]+X_test_df.shape[0]
    l = X_train_df.shape[0]
    print('train sample size %% total sample size = %.2f%%' %(100*float(l)/k))
    print('error = %.1f' %(error))

# define training set
X_train = format_train(y_df)
# splitting sample for CV : 
# take a radom part of the train sample (at least 50% of the original data size) and the last week to test results
X_train_1, X_train_CV = split_data(X_train)

# to avoid numerical problems, each values of the features is defined as a new feature
X_train_1 = pd.get_dummies(data=X_train_1, columns=['year','hour','min','dayweek'])
X_train_CV = pd.get_dummies(data=X_train_CV, columns=['year','hour','min','dayweek'])

a = X_train_1.shape[0]
b = X_train_CV.shape[0]
skf_is = ([np.arange(a-b).astype(int),np.arange(a-b,a).astype(int)])

train_test_model_clf(X_train_1, y_df, skf_is, FeatureExtractor, Regressor, False)

train data first date : 2011-01-01 00:00:00
train data last date  : 2011-04-30 01:30:00
train data range : 0 years, 3 months and 29 days

train data first date : 2011-01-01 00:00:00
train data last date  : 2011-04-25 00:30:00
train data range : 0 years, 3 months and 24 days

CV data first date : 2011-04-18 01:00:00
CV data last date  : 2011-04-25 00:30:00
CV data range : 0 years, 0 months and 6 days


TypeError: __init__() got an unexpected keyword argument 'obj'